In [ ]:
# Install and import requirements
!pip install bert-embedding --no-deps
!pip install mxnet-cu100
!pip install gluonnlp==0.8.3

import numpy as np
import mxnet as mx
from bert_embedding import BertEmbedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

# Create BERT model
ctx = mx.gpu(0)
bertmodel = BertEmbedding(model='bert_12_768_12', dataset_name='wiki_multilingual_cased', max_seq_length=200, ctx=ctx)

In [ ]:
def inference(premise, hypothesis):
    # Vectorization
    maxlen = 151 # equal to the length of the longest sentence
    test = pad_sequences(np.array(bertmodel([premise + ' | ' + hypothesis])[0][1])[None, :, :], maxlen=maxlen, dtype='float32')

    # Load network model
    model = load_model('GRU_BERT_concat.h5')
    labels = ['entailment', 'contradiction', 'neutral']
    return labels[model.predict(test).argmax(axis=1)[0]]

In [9]:
premise = 'با شهادت امام علی (ع) مرحله جدیدی از تلاش ائمه شروع شد که «دوران محنت اهل بیت» نام دارد.'
hypothesis = 'دوران سختی اهل بیت پس از شهادت امام رضا (ع) آغاز گردید.'
inference(premise, hypothesis)

'contradiction'